In [1]:
import sys
sys.path.append('..')

import lcatools
import os

In [2]:
lcia_file = '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'
spold_archive = '/data/LCI/Ecoinvent/3.2/cutoff/datasets'

catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
spold_catfile = 'ei3.2_cutoff_spold.gz'

## benchmark xlrd operation

%time X = xlrd.open_workbook(lcia_file)

In [2]:
import xlrd
import re

In [3]:
from collections import defaultdict

In [10]:
s = X.sheet_by_index(1)

In [11]:
s.name

'impact methods'

In [12]:
b = X.sheet_by_name('impact methods')

In [62]:
headings

['method',
 'category',
 'indicator',
 'name',
 'compartment',
 'subcompartment',
 'CF 3.01',
 'CF 3.1',
 'Change?',
 'note',
 'Known issue']

In [54]:
cfs = []
for row in g:
    d = dict()
    for i,h in enumerate(headings):
        d[h] = row[i]
    cfs.append(d)

In [55]:
len(cfs)

170915

In [56]:
cfs[0]

{'CF 3.01': number:1.6,
 'CF 3.1': number:1.6,
 'Change?': text:'FALSE',
 'Known issue': empty:'',
 'category': text:'acidification potential',
 'compartment': text:'air',
 'indicator': text:'average European',
 'method': text:'CML 2001',
 'name': text:'Ammonia',
 'note': text:'',
 'subcompartment': text:'low population density, long-term'}

## generate ei_lcia

In [3]:
from lcatools.providers.ecoinvent_lcia import EcoinventLcia
from lcatools.db_catalog import from_json
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [4]:
lcia_file

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

es_upstream = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/cutoff/datasets', quiet=True)

In [5]:
es_upstream = from_json(os.path.join(catalog_dir, spold_catfile))

19 new quantity entities added (19 total)
4681 new flow entities added (4681 total)
11578 new process entities added (11578 total)


In [6]:
es_upstream.processes()[135].serialize()

{'Classifications': ['ISIC rev.4 ecoinvent 3510:Electric power generation, transmission and distribution'],
 'Comment': '[This dataset is meant to replace the following datasets:]',
 'Name': 'electricity production, wind, 1-3MW turbine, onshore',
 'SpatialScope': 'LT',
 'TemporalScope': 'interval(2005-01-01, 2015-12-31)',
 'entityId': '64c029f1-4871-4a9f-8ed2-6a9f4ba4ac8a',
 'entityType': 'process',
 'exchanges': [],
 'externalId': '64c029f1-4871-4a9f-8ed2-6a9f4ba4ac8a',
 'origin': '/data/LCI/Ecoinvent/3.2/cutoff/datasets'}

In [7]:
[k['referenceUnit'].unitstring() for k in es_upstream.quantities()]

['person*km',
 'm*year',
 'm3',
 'unit',
 'ha',
 'kWh',
 'metric ton*km',
 'km*year',
 'm2*year',
 'm3*year',
 'kBq',
 'm',
 'hour',
 'km',
 'l',
 'kg',
 'm2',
 'MJ',
 'kg*day']

In [8]:
EL = EcoinventLcia(lcia_file, upstream=es_upstream, mass_quantity=es_upstream.quantity_with_unit('kg'), 
                   ns_uuid='46802ca5-8b25-398c-af10-2376adaa4623')

In [9]:
%time EL._load_xl_rows()

CPU times: user 20.5 s, sys: 70 ms, total: 20.6 s
Wall time: 20.6 s


In [10]:
len(EL._xl_rows)

170915

In [11]:
for row in EL._xl_rows:
    EL._create_flow(row)

In [12]:
qs = set()
for f in EL.flows():
    qs.add(f['referenceQuantity'].get_uuid())

In [13]:
['%s %s' % (k, EL[k]) for k in qs]

['1017f68a-f818-4f9d-a34d-a31e7386f628 LC quantity: EcoSpold Quantity m2',
 'eb955b7c-7bed-401f-9c76-5db716ca3640 LC quantity: EcoSpold Quantity m2*year',
 '487df68b-4994-4027-8fdc-a4dc298257b7 LC quantity: EcoSpold Quantity kg',
 '980b811e-3905-4797-82a5-173f5568bc7e LC quantity: EcoSpold Quantity MJ',
 'de5b3c87-0e35-4fb0-9765-4f3ba34c99e5 LC quantity: EcoSpold Quantity m3',
 '4923348e-591b-4772-b224-d19df86f04c9 LC quantity: EcoSpold Quantity kBq']

In [14]:
EL.check_counter()

0 new process entities added (0 total)
3255 new flow entities added (3255 total)
6 new quantity entities added (6 total)


In [15]:
f = EL.search('30f6b7f3-65b0-3705-b096-a2ef311315c2')[0]

In [16]:
str(f)

'Ammonia [air, lower stratosphere + upper troposphere]'

In [17]:
es_upstream.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

[]

In [18]:
EL.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

In [19]:
EL.flows()[0].__dict__

{'_characterizations': set(),
 '_d': {'CasNumber': '007447-40-7',
  'Comment': 'EcoSpold02 Flow',
  'Compartment': ['natural resource', 'in ground'],
  'Name': 'Sylvite, 25 % in sylvinite, in ground',
  'origin': '/data/LCI/Ecoinvent/3.2/cutoff/datasets'},
 '_external_ref': 'Sylvite, 25 % in sylvinite, in ground, natural resource, in ground',
 '_ref_quantity_factor': 1.0,
 '_uuid': UUID('d80610f2-df83-4e2a-9dc3-f74fced6577f'),
 'entity_type': 'flow',
 'reference_entity': <lcatools.entities.LcQuantity at 0x7fb2ba830518>}

In [20]:
EL.flows()[0].get_signature()

{'CasNumber': '007447-40-7',
 'Comment': 'EcoSpold02 Flow',
 'Compartment': ['natural resource', 'in ground'],
 'EntityType': 'flow',
 'Name': 'Sylvite, 25 % in sylvinite, in ground',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fb2ba830518>}

In [21]:
%time EL.load_all()

0 new process entities added (0 total)
0 new flow entities added (3255 total)
701 new quantity entities added (707 total)
CPU times: user 10.2 s, sys: 60 ms, total: 10.2 s
Wall time: 10.2 s


In [23]:
[ '%10.3e [] %s' % (x.value, x.quantity) 
 for x in EL['9d07940b-de52-3114-a800-9c3b26530734']._characterizations 
 if x.value is not None]

[' 3.325e-02 [] LC quantity: eco-indicator 99, (H,A), human health, respiratory effects',
 ' 2.477e-02 [] LC quantity: eco-indicator 99, (E,E), total, total',
 ' 2.477e-02 [] LC quantity: eco-indicator 99, (E,E), human health, total',
 ' 2.000e-01 [] LC quantity: EDIP, environmental impact, photochemical ozone formation, low NOx POCP',
 ' 1.000e+00 [] LC quantity: selected LCI results, air, NMVOC',
 ' 8.353e-01 [] LC quantity: USEtox, ecotoxicity, total',
 ' 1.400e+04 [] LC quantity: ecological scarcity 2013, total, total',
 ' 6.400e+04 [] LC quantity: ecological scarcity 1997, total, total',
 ' 3.325e-02 [] LC quantity: eco-indicator 99, (H,A), human health, total',
 ' 2.477e-02 [] LC quantity: eco-indicator 99, (E,E), human health, respiratory effects',
 ' 3.000e-01 [] LC quantity: EDIP, environmental impact, photochemical ozone formation, high NOx POCP',
 ' 1.800e+04 [] LC quantity: ecological scarcity 2006, total, total',
 ' 2.000e+00 [] LC quantity: EDIP, environmental impact, glo

In [27]:
EL['9d07940b-de52-3114-a800-9c3b26530734']['origin']

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

In [25]:
EL.write_to_file(os.path.join(catalog_dir, 'ei_lcia.json.gz'), gzip=True, characterizations=True, values=True)

In [28]:
EL.flows()[100].get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['water', 'ocean'],
 'EntityType': 'flow',
 'Name': 'Cobalt-60',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fb2ba830518>}

In [29]:
"""
entity generator-- I need a flow
"""

def en_gen(archive):
    for k in archive._entities.keys():
        v = archive._get_entity(k)
        if v.entity_type == 'flow':
            yield v
u = en_gen(es_upstream)

In [30]:
e = next(u)

In [31]:
next(u)['Compartment']

['natural resource', 'land']

In [32]:
e = next(u)
[e['Name']] + e['Compartment']

['Strontium', 'soil', 'agricultural']

In [33]:
e

In [37]:
unique_names = set([f['Name'] for f in EL.flows()])

In [38]:
len(unique_names)

984

In [39]:
list(unique_names)[:10]

['Metamorphous rock, graphite containing, in ground',
 'Barium-140',
 'Pirimicarb',
 'Zinc-65',
 'Sylvite, 25 % in sylvinite, in ground',
 'Phosphoric acid',
 'Methyl lactate',
 'Polychlorinated biphenyls',
 'Copper, 2.19% in sulfide, Cu 1.83% and Mo 8.2E-3% in crude ore, in ground',
 'Iodine']